
# Proteomics Pipeline: PXD011796 (NET Reference)

End-to-end Colab workflow to convert RAW files, run MSFragger/Philosopher, and export log2 protein abundance matrices for the NET reference dataset (PXD011796).


In [ ]:

# === Master Configuration ===
PROJECT_DIR = "/content/drive/MyDrive/NetsAnalysisProject"
RAW_PROTEO_DIR = f"{PROJECT_DIR}/data/raw/proteomics"
REFERENCE_DIR = f"{PROJECT_DIR}/data/raw/reference"
PXD_ID = "PXD011796"
RAW_INPUT_DIR = f"{RAW_PROTEO_DIR}/{PXD_ID}"
OUTPUT_DIR = f"{PROJECT_DIR}/data/processed/proteomics/{PXD_ID}"
MZML_DIR = f"{OUTPUT_DIR}/mzml"
WORK_DIR = f"{PROJECT_DIR}/tmp/{PXD_ID}"
TOOLS_DIR = f"{PROJECT_DIR}/tools"
MSCONVERT_BIN = "/usr/local/bin/msconvert_pwiz"
MSFRAGGER_JAR = Path('msfragger_jar.txt').read_text().strip() if Path('msfragger_jar.txt').exists() else ''
PHILOSOPHER_BIN = f"{TOOLS_DIR}/philosopher/philosopher"
FASTA_GZ = f"{REFERENCE_DIR}/UP000005640_9606.fasta.gz"
FASTA_PATH = f"{REFERENCE_DIR}/UP000005640_9606.fasta"
PROTEIN_MATRIX = f"{OUTPUT_DIR}/protein_abundance.tsv"
METADATA_PATH = f"{OUTPUT_DIR}/metadata.tsv"
UNIPROT_MAP_PATH = f"{OUTPUT_DIR}/uniprot_to_hgnc.tsv"


In [ ]:

# Mount Drive and ensure directories
try:
    import google.colab  # type: ignore
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
except ModuleNotFoundError:
    print('[info] Running outside Colab; ensure PROJECT_DIR is accessible.')

from pathlib import Path

for path in [Path(PROJECT_DIR), Path(RAW_INPUT_DIR), Path(OUTPUT_DIR), Path(MZML_DIR), Path(WORK_DIR), Path(TOOLS_DIR)]:
    path.mkdir(parents=True, exist_ok=True)

print(f"[setup] RAW files expected under: {RAW_INPUT_DIR}")
print(f"[setup] Processed outputs -> {OUTPUT_DIR}")


In [ ]:
# Install tooling: micromamba + ProteoWizard shim, MSFragger (bioconda), Philosopher
!apt-get update -qq
!apt-get install -yqq wget aria2

import subprocess
import tarfile
import requests
import os
from pathlib import Path

TOOLS_DIR_PATH = Path(TOOLS_DIR)
TOOLS_DIR_PATH.mkdir(parents=True, exist_ok=True)

MICROMAMBA_BIN = Path.home() / 'bin' / 'micromamba'
ENV_NAME = 'pwiz'
SHIM_PATH = Path('/usr/local/bin/msconvert_pwiz')


def download(url: str, dest: Path):
    if dest.exists():
        print(f"[cache] {dest.name} already present")
        return
    dest.parent.mkdir(parents=True, exist_ok=True)
    print(f"[download] {url}")
    with requests.get(url, stream=True, timeout=120) as r:
        r.raise_for_status()
        with open(dest, 'wb') as fh:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    fh.write(chunk)

# Install micromamba if missing
if not MICROMAMBA_BIN.exists():
    tarball = TOOLS_DIR_PATH / 'micromamba.tar.bz2'
    download('https://micromamba.snakepit.net/api/micromamba/linux-64/latest', tarball)
    (Path.home() / 'bin').mkdir(exist_ok=True)
    with tarfile.open(tarball, 'r:bz2') as tf:
        members = [m for m in tf.getmembers() if m.name.startswith('bin/')]
        tf.extractall(path=Path.home(), members=members, filter='data')
    MICROMAMBA_BIN.chmod(0o755)
else:
    print('[cache] micromamba already installed')

# Ensure proteowizard + msfragger packages are installed
create_cmd = [str(MICROMAMBA_BIN), 'create', '-y', '-n', ENV_NAME,
              '-c', 'conda-forge', '-c', 'bioconda', 'proteowizard', 'msfragger']
install_cmd = [str(MICROMAMBA_BIN), 'install', '-y', '-n', ENV_NAME,
               '-c', 'conda-forge', '-c', 'bioconda', 'proteowizard', 'msfragger']
ret = subprocess.run(create_cmd)
if ret.returncode != 0:
    subprocess.run(install_cmd, check=True)

# Verify msfragger is present
subprocess.run([str(MICROMAMBA_BIN), 'run', '-n', ENV_NAME, 'bash', '-lc', 'conda list msfragger'], check=True)

# Verify msconvert availability
subprocess.run([str(MICROMAMBA_BIN), 'run', '-n', ENV_NAME, 'msconvert', '--help'],
               check=True, stdout=subprocess.DEVNULL)

# Create shim script for msconvert
shim_script = f"""#!/usr/bin/env bash
set -e
{MICROMAMBA_BIN} run -n {ENV_NAME} msconvert "$@"
"""
SHIM_PATH.write_text(shim_script)
SHIM_PATH.chmod(0o755)

# Resolve real msconvert path
def resolve_msconvert_path() -> Path:
    result = subprocess.run([str(MICROMAMBA_BIN), 'run', '-n', ENV_NAME, 'which', 'msconvert'],
                            check=True, capture_output=True, text=True)
    return Path(result.stdout.strip())

msconvert_real = resolve_msconvert_path()

globals()['MSCONVERT_BIN'] = str(SHIM_PATH)
print(f"[check] msconvert shim at {MSCONVERT_BIN}")
print(f"[check] real msconvert binary at {msconvert_real}")

# Determine MSFragger jar path from environment
conda_prefix = subprocess.check_output([
    str(MICROMAMBA_BIN), 'run', '-n', ENV_NAME, 'bash', '-lc', 'printf %s "$CONDA_PREFIX"'
], text=True).strip()
msfragger_dir = Path(conda_prefix) / 'share' / 'msfragger'
jar_candidates = sorted(msfragger_dir.glob('MSFragger-*.jar'))
if not jar_candidates:
    raise FileNotFoundError(f'MSFragger jar not found under {msfragger_dir}. Ensure msfragger package provides the jar (conda-forge/bioconda).')
msfragger_jar_path = jar_candidates[-1]

globals()['MSFRAGGER_JAR'] = str(msfragger_jar_path)
print(f"[check] MSFragger jar at {MSFRAGGER_JAR}")

# Philosopher installer (GitHub release tarball)
PHILOSOPHER_PATH = Path(PHILOSOPHER_BIN)
if not PHILOSOPHER_PATH.exists():
    tgz_path = TOOLS_DIR_PATH / 'philosopher_v5.2.0_linux_amd64.tar.gz'
    download('https://github.com/Nesvilab/philosopher/releases/download/v5.2.0/philosopher_v5.2.0_linux_amd64.tar.gz', tgz_path)
    subprocess.run(['tar', '-xzf', str(tgz_path), '-C', str(TOOLS_DIR_PATH)], check=True)
else:
    print('[cache] Philosopher already installed')

print(f"[check] Philosopher binary found: {PHILOSOPHER_PATH.exists()}")


In [ ]:

# Helper functions for proteomics workflow
import subprocess
import gzip
import shutil
import pandas as pd
import numpy as np
import time
import requests
from pathlib import Path
from typing import List


def run_cmd(cmd: List[str], cwd: Path | None = None):
    print('[cmd]', ' '.join(cmd))
    subprocess.run(cmd, check=True, cwd=str(cwd) if cwd else None)


def ensure_fasta() -> Path:
    fasta_path = Path(FASTA_PATH)
    if fasta_path.exists():
        return fasta_path
    gz_path = Path(FASTA_GZ)
    if not gz_path.exists():
        raise FileNotFoundError(f"FASTA gz not found: {gz_path}")
    with gzip.open(gz_path, 'rt') as fin, open(fasta_path, 'w') as fout:
        shutil.copyfileobj(fin, fout)
    print(f"[fasta] Decompressed {gz_path} -> {fasta_path}")
    return fasta_path


def convert_raw_to_mzml():
    raw_dir = Path(RAW_INPUT_DIR)
    mzml_dir = Path(MZML_DIR)
    mzml_dir.mkdir(parents=True, exist_ok=True)
    raw_files = sorted(raw_dir.glob('*.raw'))
    if not raw_files:
        raise FileNotFoundError(f"No RAW files found in {raw_dir}")
    for raw in raw_files:
        out_file = mzml_dir / (raw.stem + '.mzML')
        if out_file.exists():
            print(f"[skip] {out_file.name} already present")
            continue
        run_cmd([
            MSCONVERT_BIN,
            str(raw),
            '--mzML',
            '--filter', 'peakPicking true 1-',
            '--outfile', out_file.name,
            '--outdir', str(mzml_dir)
        ])
    print(f"[msconvert] Completed {len(raw_files)} conversions")


def write_msfragger_params(fasta_path: Path) -> Path:
    params_path = Path(WORK_DIR) / 'msfragger.params'
    params_path.parent.mkdir(parents=True, exist_ok=True)
    params_path.write_text(
        f'''num_threads = 16
precursor_true_tolerance = 15
enzyme = trypsin
isotope_error = 0/1/2
allowed_missed_cleavage = 2
fragment_bin_tol = 0.02
precursor_mass_lower = -20
precursor_mass_upper = 20
precursor_mass_units = ppm
fragment_mass_units = Dalton
search_enzyme_name = trypsin
protein_database = {fasta_path}
add_Cterm_peptide = 0.0
add_Nterm_peptide = 0.0
add_G_glycine = 0.0
add_C_cysteine = 57.021464
variable_mod_01 = 15.994915 M 100.0
variable_mod_02 = 15.994915 P 10.0
variable_mod_03 = 0.984016 NQ 1.0
localize_delta_mass = 1
replace_missing = 1
zero_bin_accept_expect = 0
remove_precursor_peak = 1
isotope_error_correction = 1
calculate_pq_peptide_probability = 1
'''
    )
    return params_path


def run_msfragger(params_path: Path):
    mzml_files = sorted(Path(MZML_DIR).glob('*.mzML'))
    if not mzml_files:
        raise FileNotFoundError('No mzML files available for MSFragger')
    cmd = ['java', '-Xmx24G', '-jar', MSFRAGGER_JAR, str(params_path)] + [str(f) for f in mzml_files]
    run_cmd(cmd, cwd=Path(WORK_DIR))


def run_philosopher(fasta_path: Path) -> Path:
    workspace = Path(WORK_DIR) / 'philosopher'
    workspace.mkdir(parents=True, exist_ok=True)
    run_cmd([PHILOSOPHER_BIN, 'workspace', '--init'], cwd=workspace)
    run_cmd([PHILOSOPHER_BIN, 'database', '--annotate', str(fasta_path)], cwd=workspace)

    pepxml_files = sorted(Path(WORK_DIR).glob('*.pepXML')) or sorted(Path(WORK_DIR).glob('*.pep.xml'))
    if not pepxml_files:
        raise FileNotFoundError('No pepXML files produced by MSFragger')

    for pepxml in pepxml_files:
        run_cmd([PHILOSOPHER_BIN, 'peptideprophet', '--ppm', '--nonparam', '--expectscore', str(pepxml)], cwd=workspace)

    interact_files = sorted(workspace.glob('interact-*.pep.xml'))
    run_cmd([PHILOSOPHER_BIN, 'proteinprophet'] + [str(f) for f in interact_files], cwd=workspace)
    run_cmd([PHILOSOPHER_BIN, 'filter', '--sequential', '--picked', '--peptide', '--prot'], cwd=workspace)
    run_cmd([PHILOSOPHER_BIN, 'label-free'] + [str(f) for f in interact_files], cwd=workspace)
    run_cmd([PHILOSOPHER_BIN, 'report', '--msstats'], cwd=workspace)
    return workspace


def load_protein_table(workspace: Path) -> pd.DataFrame:
    candidates = list(workspace.glob('*.protein.tsv'))
    if not candidates:
        raise FileNotFoundError('Philosopher protein report not found')
    return pd.read_csv(candidates[0], sep='	')


def uniprot_to_gene(uniprot_ids: List[str]) -> pd.DataFrame:
    ids = [uid for uid in set(uniprot_ids) if isinstance(uid, str) and uid]
    if not ids:
        return pd.DataFrame(columns=['UniProt', 'Gene'])
    job = requests.post('https://rest.uniprot.org/idmapping/run', data={
        'from': 'UniProtKB_AC-ID',
        'to': 'Gene_Name',
        'ids': ','.join(ids)
    }).json()
    job_id = job['jobId']
    status_url = f'https://rest.uniprot.org/idmapping/status/{job_id}'
    result_url = f'https://rest.uniprot.org/idmapping/stream/{job_id}'
    while True:
        status = requests.get(status_url).json()
        if status.get('jobStatus') == 'FINISHED':
            break
        if status.get('jobStatus') == 'FAILED':
            raise RuntimeError('UniProt mapping failed')
        time.sleep(3)
    rows = []
    for entry in requests.get(result_url).json().get('results', []):
        gene = entry.get('to')
        if gene:
            rows.append({'UniProt': entry['from'], 'Gene': gene.split(';')[0]})
    return pd.DataFrame(rows).drop_duplicates()


def build_matrix(df: pd.DataFrame) -> pd.DataFrame:
    intensity_cols = [c for c in df.columns if 'Intensity' in c]
    if not intensity_cols:
        raise ValueError('No intensity columns detected in protein report')
    df = df.copy()
    df['PrimaryAcc'] = df['Protein'].str.split(';').str[0]
    mapping = uniprot_to_gene(df['PrimaryAcc'].tolist())
    df = df.merge(mapping, how='left', left_on='PrimaryAcc', right_on='UniProt')
    df['Gene'] = df['Gene'].fillna(df['PrimaryAcc'])
    df = df.drop_duplicates(subset=['Gene']).set_index('Gene')
    matrix = np.log2(df[intensity_cols].replace(0, np.nan))
    matrix.to_csv(PROTEIN_MATRIX, sep='	')
    if not mapping.empty:
        mapping.to_csv(UNIPROT_MAP_PATH, sep='	', index=False)
    else:
        df[['PrimaryAcc']].rename(columns={'PrimaryAcc': 'UniProt'}).assign(Gene=df.index).to_csv(UNIPROT_MAP_PATH, sep='	', index=False)
    return matrix


def derive_metadata(matrix: pd.DataFrame) -> pd.DataFrame:
    records = []
    for sample in matrix.columns:
        clean = sample.replace('.raw', '').replace('.mzML', '')
        tokens = clean.split('_')
        disease = 'Unknown'
        stimulus = 'NA'
        timepoint = 'NA'
        for token in tokens:
            upper = token.upper()
            if upper.startswith('SLE'):
                disease = 'SLE'
            elif upper.startswith('RA'):
                disease = 'RA'
            elif upper.startswith('NC'):
                disease = 'Healthy'
            if upper in {'PMA', 'A23', 'A23187'}:
                stimulus = upper
            if upper.endswith('H') and upper[:-1].isdigit():
                timepoint = upper
        records.append({
            'sample_id': sample,
            'disease': disease,
            'stimulus': stimulus,
            'timepoint': timepoint
        })
    meta = pd.DataFrame(records)
    meta.to_csv(METADATA_PATH, sep='	', index=False)
    return meta


### Step 1: Prepare FASTA

In [ ]:
fasta_path = ensure_fasta(); fasta_path

### Step 2: Convert RAW files to mzML

In [ ]:
convert_raw_to_mzml()

### Step 3: Run MSFragger search

In [ ]:
params_file = write_msfragger_params(ensure_fasta()); run_msfragger(params_file)

### Step 4: Run Philosopher post-processing

In [ ]:
workspace_dir = run_philosopher(ensure_fasta()); workspace_dir

### Step 5: Build protein matrix and metadata

In [ ]:
protein_df = load_protein_table(workspace_dir); matrix = build_matrix(protein_df); metadata = derive_metadata(matrix); matrix.shape

In [ ]:
print(f'Protein matrix saved to: {PROTEIN_MATRIX}')
print(f'Metadata saved to: {METADATA_PATH}')
print(f'UniProt mapping saved to: {UNIPROT_MAP_PATH}')